In [1]:
import json
import pandas as pd
import altair as alt

In [2]:
# Load the sales data from the JSON file
with open("retail_sales_dataset.json", "r") as f:
    sales_data = json.load(f)

# Convert data to DataFrame for processing
df = pd.DataFrame(sales_data)

# Ensure correct column names match dataset
if "Product Category" not in df.columns or "Quantity" not in df.columns:
    raise ValueError("Column names do not match expected structure. Check JSON file.")

# Define Vega-Lite chart schema with FIXES
chart_schema = {
    "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
    "mark": "bar",
    "encoding": {
        "x": {
            "field": "Product Category",
            "type": "nominal",
            "title": "Product Category",
            "sort": {"field": "Total Quantity Sold", "order": "descending"}
        },
        "y": {
            "field": "Total Quantity Sold",
            "type": "quantitative",
            "title": "Total Quantity Sold"
        },
        "color": {
            "field": "Product Category",
            "type": "nominal",
            "legend": {"title": "Product Category"}
        }
    },
    "transform": [
        {
            "aggregate": [{"op": "sum", "field": "Quantity", "as": "Total Quantity Sold"}],
            "groupby": ["Product Category"]
        },
        {
            "window": [{"op": "rank", "as": "rank"}],
            "frame": [None, 0],  # Fix missing frame
            "sort": [{"field": "Total Quantity Sold", "order": "descending"}]
        },
        {
            "filter": "datum.rank <= 5"  # Fix invalid filter notation
        }
    ]
}

# Inject sales data into the Vega-Lite schema
chart_schema["data"] = {"values": df.to_dict(orient="records")}

# Convert Vega-Lite JSON schema to Altair chart
chart = alt.Chart.from_json(json.dumps(chart_schema))

# Display the chart in Jupyter Notebook
chart.display()

alt.Chart(...)